In [ ]:
import pandas as pd
import plotly.express as px
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import ipywidgets as widgets
from IPython.display import display
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import os
import io
import time
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from PIL import Image as PILImage
import pandas as pd
#import pandas as pd

In [ ]:

def make_question(question_text, options, correct_answer):
    # Widgets
    dropdown = widgets.Dropdown(
        options=[""] + options,   # ensure "" starts blank
        value="",
        description="Answer:",
        style={'description_width': 'initial'}
    )

    button = widgets.Button(
        description="Check Answer",
        button_style="info"
    )

    output = widgets.Output()

    # Callback (closes over dropdown, output, correct_answer)
    def check_answer(b):
        with output:
            clear_output()
            if dropdown.value == "":
                print("Please select an answer!")
            elif dropdown.value == correct_answer:
                print("✅ Correct!")
            else:
                print(f"❌ Incorrect. The correct answer is {correct_answer}.")

    button.on_click(check_answer)

    # Display
    display(widgets.HTML(f"<h3>{question_text}</h3>"))
    display(dropdown, button, output)

In [ ]:
# Load the audio file
audio = widgets.Audio.from_file("simcity_loop_snes.wav")
audio.loop = True
audio.autoplay = False   # browsers usually block autoplay until first click
audio.layout = widgets.Layout(width="200px")  # shrink the player bar

# Label + audio side by side
box = widgets.HBox([
    widgets.Label("", layout=widgets.Layout(width="0px")),
    audio
])

display(box)
#src = Path("Soundtrack\loop_1min.wav")

<br><center>[Return to Planning](./02_Planning.ipynb)</center>

<h1>Now it is Time For You to Design the Pipeline Entering the Wastewater Treatment Plant!</h1>
Now that you have sucessfully calculated the flowrate entering the wastewater treatment plant, it is now time for you to size the piping entering the wastewater treatment plant using these flows!
In pipeline design of interceptors we typically want the depth of water to the diameter of the pipe to be less than 75% full at peak flow conditions.  This is commonly called 75% d/D!  We also want the pipe to scour so that solids do not settle in the pipe, and to do this engineers typically design their pipes to have at least 2 feet per second at peak flow conditions every day.  Finally, we also do not want the wastewater to go to fast through the pipe as it will degrade the pipe itself, and a typical parameter for this is that we do not want it to exceed 10 feet per second at peak flow conditions.

We check these conditions at the beginning and end of the design life cycle for the pipeline.  This means that your pipe needs to scour at least once per day in 1955 (> 2 feet per second velocity in the pipe) and it needs to be less than 75% d/D in 1980!

We will use a simple peaking factor of 2.25 over the average flow!

To aid you in this process we will allow you to use modern modeling methods where all you need to do is choose the diameter you want!  You have a network of four pipes leading to the Wastewater Treatment Plant!

In [ ]:
#Read the Pipes
Pipes = pd.read_csv('Pipes.csv')
#Pipes

,Pipe Size,Flow Rate,Max Velocity,Max d/D
0,12,2.7,6.0,100
1,12,6.0,8.3,100
2,12,18.0,9.6,100
3,18,2.7,6.3,45
4,18,6.0,7.5,75
5,18,18.0,11.0,100
6,24,2.7,5.8,30
7,24,6.0,7.3,46
8,24,18.0,12.0,100
9,36,2.7,5.5,18


In [ ]:
#This is a manning's pipe calculator for rectangular channels.  Delete the comments to use it.
#def calculate_flow_rate_rectangular(n, b, y, S):
    #"""
    #Calculates the flow rate in a rectangular open channel using Manning's equation.

    #Args:
        #n (float): Manning's roughness coefficient.
        #b (float): Channel bottom width (ft).
        #y (float): Water depth (ft).
        #S (float): Channel slope (ft/ft).

    #Returns:
        #float: Flow rate (cfs).
    #"""
    #A = b * y  # Cross-sectional area
    #P = b + 2 * y  # Wetted perimeter
    #R = A / P  # Hydraulic radius
    #Q = (1.49 / n) * A * (R**(2/3)) * (S**(1/2))*0.64631688969744
    #return Q

# Example usage:
#n_value = 0.013  # Concrete channel
#bottom_width = 4.0 # ft
#water_depth = 2.0  # ft
#channel_slope = 0.001 # ft/ft

#flow_rate = calculate_flow_rate_rectangular(n_value, bottom_width, water_depth, channel_slope)
#print(f"Calculated Flow Rate: {flow_rate:.2f} MGD")

In [ ]:
import os
import io
import time
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from PIL import Image as PILImage
import pandas as pd
#import pandas as pd

# --- Your table (already loaded outside) ---
# Pipes = pd.read_csv('SWMM/pipes.csv')

# --- Helper: build candidate filenames and return any that exist ---
def find_images_for(size_in, flow_mgd, base_dir=".", stems=("profile",)):
    """
    Try common filename patterns and return a list of existing paths.
    Patterns tried (in priority order), per stem:
      {size}-inch-{flow}MGD_cropped.png
      {size}-inch-{flow}MGD.png
    """
    flow_str = str(flow_mgd).rstrip("0").rstrip(".")  # 6.0 -> "6", 2.7 -> "2.7"
    patterns = [
        f"{size_in}-inch-{flow_str}MGD_cropped.png",
        f"{size_in}-inch-{flow_str}MGD.png",
    ]
    out = []
    for p in patterns:
        full = os.path.join(base_dir, p)
        if os.path.exists(full):
            out.append(full)
    return out

# --- Helper: display a PIL-resized image at 50% scale ---
def display_resized_50p(path):
    try:
        img = PILImage.open(path)
        half = (max(1, img.width // 2), max(1, img.height // 2))
        img_small = img.resize(half)
        buf = io.BytesIO()
        img_small.save(buf, format="PNG")
        buf.seek(0)
        display(widgets.Image(value=buf.read(), format="png"))
    except Exception as e:
        display(HTML(f"<div style='color:#b00'>⚠️ Could not load {path}: {e}</div>"))

# --- The widget ---
def make_pipe_selector_with_table(
    title="Select a pipe size and flow rate to simulate:",
    sizes=("12","18","24","36"),
    flows=("2.7","6","18"),
    df=None
):
    if df is None:
        raise ValueError("Please pass your Pipes DataFrame via df=...")

    # Dropdowns
    dd_size = widgets.Dropdown(
        options=[""] + list(sizes),
        value="",
        description="Pipe Size (in):",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width="220px")
    )
    dd_flow = widgets.Dropdown(
        options=[""] + [f"{f}" for f in flows],
        value="",
        description="Flow Rate (MGD):",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width="220px")
    )

    btn = widgets.Button(
        description="Simulate",
        icon="play",
        button_style="info"
    )

    out = widgets.Output()

    def on_click(_):
        with out:
            clear_output()

            if not dd_size.value or not dd_flow.value:
                print("Please select both a pipe size and a flow rate.")
                return

            # Parse selections
            size_sel = dd_size.value            # e.g. "12"
            flow_sel = dd_flow.value.split()[0] # "2.7", "6.0", "18.0"
            flow_num = float(flow_sel)

            # Show mini "simulate" delay
            print("⏳ Simulating... Please Wait...")
            time.sleep(2)
            # --- Table row (Max Velocity & Max d/D) ---
            try:
                mask = (df["Pipe Size"].astype(str) == str(size_sel)) & (df["Flow Rate"].astype(float) == flow_num)
                row = df.loc[mask, ["Pipe Size", "Flow Rate", "Max Velocity", "Max d/D"]].copy()
                if row.empty:
                    display(HTML("<b>No matching row in Pipes table for this selection.</b>"))
                else:
                    # Pretty formatting
                    row = row.rename(columns={
                        "Pipe Size": "Pipe (in)",
                        "Flow Rate": "Flow Rate (MGD)",
                        "Max Velocity": "Max Velocity (ft/s)",
                        "Max d/D": "Max d/D (%)"
                    })
                    # Display the single-row table nicely
                    display(HTML("<h4>Hydraulic Summary</h4>"))
                    display(row.style.format({
                        "Flow Rate(MGD)": "{:.1f}",
                        "Max Velocity (ft/s)": "{:.1f}",
                        "Max d/D (%)": "{:.0f}",
                    }).set_properties(**{"text-align": "center"}).hide(axis="index"))
            except Exception as e:
                display(HTML(f"<div style='color:#b00'>⚠️ Could not render table: {e}</div>"))
            # --- Images ---
            images = find_images_for(size_sel, flow_num)
            if images:
                for img_path in images:
                    display(HTML("<h4>Hydraulic Profile</h4>"))
                    #display(HTML(f"<b>{os.path.basename(img_path)}</b>"))
                    display_resized_50p(img_path)
            else:
                display(HTML("<b>No image found for this combination.</b> "
                             "Expected something like "
                             f"<code>{size_sel}-inch-{flow_sel}MGD_cropped.png</code>"))

            

    btn.on_click(on_click)

    # Layout
    header = widgets.HTML(f"<h3>{title}</h3>")
    controls = widgets.HBox([dd_size, dd_flow, btn], layout=widgets.Layout(gap="12px"))
    display(header, controls, out)

# ---- Use it ----
make_pipe_selector_with_table(
    df=Pipes,
    sizes=("12","18","24","36"),
    flows=("2.7","6.0","18.0"),
)


Now it is quiz time since you used your handy dandy pipe simulator tool for the pipe!

In [ ]:
make_question(
    "What is the best pipe diameter for the pipe above?",
    ["12-inch", "18-inch", "24-inch", "36-inch"],
    "36-inch"
)